# Join in Spark
This notebook explains a simple join operation in Spark. Code here is a solution to a programming assignment in the Coursera course `Hadoop Platform and Application Framework`. 

#### Simple `join` operation
Dataset for this section can be found in the repository. `join1_FileA.txt` and `join1_FileA.txt` are the relevant files. These are 2 different wordcount datasets. We'll be performing a join on these two files. The content of the file isn't super clear to me. So, I'm not including the description here. Anyways, since the dataset is really small, that wouldn't be a hindrance in understanding the results. 

In [32]:
def split_fileA(line):
        # split the input line in word and count on the comma
        key_value = line.split(",")
        word = key_value[0]
        # turn the count to an integer  
        count = int(key_value[1])
        return (word, count)

In [33]:
def split_fileB(line):
        # split the input line into word, date and count_string
        key_value = line.split(",")
        count_string = key_value[1]
        date_word = key_value[0].split(" ")
        date = date_word[0]
        word = date_word[1] 
        return (word, date + " " + count_string)

In [42]:
fileA = sc.textFile("./data/simple_join/join1_FileA.txt")
fileB = sc.textFile("./data/simple_join/join1_FileB.txt")

In [43]:
fileA.collect()

[u'able,991', u'about,11', u'burger,15', u'actor,22']

In [44]:
fileB.collect()

[u'Jan-01 able,5',
 u'Feb-02 about,3',
 u'Mar-03 about,8',
 u'Apr-04 able,13',
 u'Feb-22 actor,3',
 u'Feb-23 burger,5',
 u'Mar-08 burger,2',
 u'Dec-15 able,100']

In [45]:
# Test if the split_fileA() returns a desired output
test_line = "able,991"
split_fileA(test_line)

('able', 991)

In [46]:
fileA_data = fileA.map(split_fileA)
fileA_data.collect()

[(u'able', 991), (u'about', 11), (u'burger', 15), (u'actor', 22)]

In [47]:
fileB_data = fileB.map(split_fileB)
fileB_data.collect()

[(u'able', u'Jan-01 5'),
 (u'about', u'Feb-02 3'),
 (u'about', u'Mar-03 8'),
 (u'able', u'Apr-04 13'),
 (u'actor', u'Feb-22 3'),
 (u'burger', u'Feb-23 5'),
 (u'burger', u'Mar-08 2'),
 (u'able', u'Dec-15 100')]

In [48]:
fileB_joined_fileA = fileB_data.join(fileA_data)

In [49]:
fileB_joined_fileA.collect()

[(u'able', (u'Jan-01 5', 991)),
 (u'able', (u'Apr-04 13', 991)),
 (u'able', (u'Dec-15 100', 991)),
 (u'burger', (u'Feb-23 5', 15)),
 (u'burger', (u'Mar-08 2', 15)),
 (u'about', (u'Feb-02 3', 11)),
 (u'about', (u'Mar-03 8', 11)),
 (u'actor', (u'Feb-22 3', 22))]

#### Advanced `join` operation
This too is a solution to the assignment in the aforesaid course. Dataset can be found in the repository in `./data/advanced_join`. Dataset consists of a bunch of gennum and genchan files. The gennum files contain show names and their viewers, genchan files contain show names and their channel. We want to find out the total number of viewers across all shows for the channel BAT.

In [51]:
show_views_file = sc.textFile("./data/advanced_join/join2_gennum?.txt")

In [52]:
show_views_file.take(2)

[u'Hourly_Sports,21', u'Hot_Talking,44']

In [57]:
show_channel_file = sc.textFile("./data/advanced_join/join2_genchan?.txt")
show_channel_file.take(2)

[u'Hourly_Sports,DEF', u'Hot_Cooking,XYZ']

In [53]:
def split_show_views(line):
        show_views = line.split(",")
        show = show_views[0]
        views = show_views[1]
        return (show, views)

In [54]:
def split_show_channel(line):
        show_channel = line.split(",")
        show = show_views[0]
        channel = show_views[1]
        return (show, channel)

In [55]:
def extract_channel_views(show_views_channel): 
        channel = show_views_channel[1][0]
        views = show_views_channel[1][1]
        return (channel, views)

In [ ]:
def sum_channel_views(a, b):
        channel_views = int(a) + int(b)
        return channel_views

In [59]:
show_views = show_views_file.map(split_show_views)
show_channel = show_channel_file.map(split_show_channel)

In [60]:
joined_dataset = show_channel.join(show_views)

Traceback (most recent call last):
  File "/home/abhishek/spark-2.2.0-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 148, in dump
    return Pickler.dump(self, obj)
  File "/usr/lib/python2.7/pickle.py", line 224, in dump
    self.save(obj)
  File "/usr/lib/python2.7/pickle.py", line 286, in save
    f(self, obj) # Call unbound method with explicit self
  File "/usr/lib/python2.7/pickle.py", line 568, in save_tuple
    save(element)
  File "/usr/lib/python2.7/pickle.py", line 286, in save
    f(self, obj) # Call unbound method with explicit self
  File "/home/abhishek/spark-2.2.0-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 255, in save_function
    self.save_function_tuple(obj)
  File "/home/abhishek/spark-2.2.0-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 292, in save_function_tuple
    save((code, closure, base_globals))
  File "/usr/lib/python2.7/pickle.py", line 286, in save
    f(self, obj) # Call unbound method with explicit self
  File "/usr/lib/python2.7/pickle

PicklingError: Could not serialize object: Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.